## About Dataset
Context This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content 5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset. Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.
Acknowledgements This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

### Inspiration

Sentiment analysis on reviews.
Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
Fake reviews/ outliers.
Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
Any other interesting analysis

### Best Practises
- Preprocessing And Cleaning
- Train Test Split
- BOW,TFIDF,Word2vec
- Train ML algorithms

In [33]:
import pandas as pd

data = pd.read_csv("../datasets/all_kindle_review.csv")
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [34]:
df = data[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [35]:
df.shape

(12000, 2)

In [36]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [37]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [38]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [39]:
df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)

C:\Users\ZE634LD\AppData\Local\Temp\ipykernel_19680\195494786.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)


In [40]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [41]:
# cleaning data

df['reviewText'] = df['reviewText'].str.lower()
df.head()

C:\Users\ZE634LD\AppData\Local\Temp\ipykernel_19680\1472407803.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].str.lower()


,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [42]:
import re
from bs4 import BeautifulSoup
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemma = WordNetLemmatizer()

## Removing special characters
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub('[^A-Z a-z 0-9-]+',"",x))
## Remove url 
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove the stopwords and apply lemmatization
df['reviewText'] = df['reviewText'].apply(lambda x: " " .join([lemma.lemmatize(y) for y in x.split() if y not in stopwords.words('english')]))

C:\Users\ZE634LD\AppData\Local\Temp\ipykernel_19680\3184267745.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].apply(lambda x: re.sub('[^A-Z a-z 0-9-]+',"",x))
C:\Users\ZE634LD\AppData\Local\Temp\ipykernel_19680\3184267745.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
C:\Users\ZE634LD\AppData\Local

In [43]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [44]:
X = df['reviewText']
y = df['rating']

In [45]:
# train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [46]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((9600,), (9600,), (2400,), (2400,))

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=2500, ngram_range=(1,2))
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

In [48]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [49]:
X_test_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=2500, ngram_range=(1,2))
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [54]:
X_train_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [55]:
X_test_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [57]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

nb = MultinomialNB()

nb.fit(X_train_bow, y_train)
y_pred_bow = nb.predict(X_test_bow)

print(f"classification report of bow for naive bayes is : {classification_report(y_test,y_pred_bow)}")
print(f"accuracy score of bow for naive bayes is : {accuracy_score(y_test,y_pred_bow)}")
print(f"confusion matrix of bow for naive bayes is : {confusion_matrix(y_test,y_pred_bow)}")

classification report of bow for naive bayes is :               precision    recall  f1-score   support

           0       0.69      0.79      0.74       779
           1       0.89      0.83      0.86      1621

    accuracy                           0.82      2400
   macro avg       0.79      0.81      0.80      2400
weighted avg       0.83      0.82      0.82      2400

accuracy score of bow for naive bayes is : 0.8191666666666667
confusion matrix of bow for naive bayes is : [[ 617  162]
 [ 272 1349]]


In [58]:
nb.fit(X_train_tfidf, y_train)
y_pred_tfidf = nb.predict(X_test_tfidf)

print(f"classification report of bow for naive bayes is : {classification_report(y_test,y_pred_tfidf)}")
print(f"accuracy score of bow for naive bayes is : {accuracy_score(y_test,y_pred_tfidf)}")
print(f"confusion matrix of bow for naive bayes is : {confusion_matrix(y_test,y_pred_tfidf)}")

classification report of bow for naive bayes is :               precision    recall  f1-score   support

           0       0.81      0.57      0.67       779
           1       0.82      0.94      0.87      1621

    accuracy                           0.82      2400
   macro avg       0.82      0.75      0.77      2400
weighted avg       0.82      0.82      0.81      2400

accuracy score of bow for naive bayes is : 0.8179166666666666
confusion matrix of bow for naive bayes is : [[ 447  332]
 [ 105 1516]]
